<a href="https://colab.research.google.com/github/luctiz/Grupo43-TP2/blob/master/TensorFLow_RedesNeuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format





In [0]:
test =  pd.read_csv("https://metadata.fundacionsadosky.org.ar/media/navent/test.csv", sep=",",parse_dates = ['fecha'])
train = pd.read_csv("https://metadata.fundacionsadosky.org.ar/media/navent/train.csv", sep=",",parse_dates = ['fecha'])

In [335]:
!pip install category_encoders


In [0]:
#A los nulls los relleno con el promedio de metroscubiertos de todas las publicaciones
train['metroscubiertos']=train['metroscubiertos'].fillna(round(train['metroscubiertos'].mean()))
test['metroscubiertos']=test['metroscubiertos'].fillna(round(test['metroscubiertos'].mean()))
#Luego convierto metroscubiertos a  np.int16
train['metroscubiertos']=train['metroscubiertos'].astype(np.int16)
test['metroscubiertos']=test['metroscubiertos'].astype(np.int16)

#A los nulls los relleno con el promedio de metrostotales de todas las publicaciones
train['metrostotales']=train['metrostotales'].fillna(round(train['metrostotales'].mean()))
test['metrostotales']=test['metrostotales'].fillna(round(test['metrostotales'].mean()))
#Luego convierto metrostotales a  np.int16
train['metrostotales']=train['metrostotales'].astype(np.int16)
test['metrostotales']=test['metrostotales'].astype(np.int16)

#El precio promedio de las publicaciones que no tienen datos de cant de habitaciones esta entre el precio
#promedio de las que tienen 2 y 3 habitaciones
#Entonces voy a asumir que tienen 3 habitaciones, ya que es el valor al cual el precio mas se les asemeja.
train['habitaciones']=train['habitaciones'].fillna(3)
test['habitaciones']=test['habitaciones'].fillna(3)
#Luego convierto a np.int8
train['habitaciones']=train['habitaciones'].astype(np.int8)
test['habitaciones']=test['habitaciones'].astype(np.int8)

#A los que no tienen datos de antiguedad los relleno con la antiguedad promedio (es 8)
train['antiguedad']=train['antiguedad'].fillna(round(train['antiguedad'].mean()))
test['antiguedad']=test['antiguedad'].fillna(round(train['antiguedad'].mean()))
#Luego convierto a np.uint8
train['antiguedad']=train['antiguedad'].astype(np.uint8)
test['antiguedad']=test['antiguedad'].astype(np.uint8)

#El precio promedio de los que tienen NULL en garage es parecido al de los que tienen 2.
#Relleno los nulls de esta columna con un 2.
train['garages']=train['garages'].fillna(2)
test['garages']=test['garages'].fillna(2)
#convierto a np.int8
train['garages']=train['garages'].astype(np.int8)
test['garages']=test['garages'].astype(np.int8)

#El precio promedio de los que tienen NULL en banos es mas parecido al de los que tienen 2.
#Relleno los nulls de esta columna con un 2.
train['banos']=train['banos'].fillna(2)
test['banos']=test['banos'].fillna(2)
#convierto a np.int8
train['banos']=train['banos'].astype(np.int8)
test['banos']=test['banos'].astype(np.int8)

train['gimnasio']=train['gimnasio'].astype(np.int8)
train['usosmultiples']=train['usosmultiples'].astype(np.int8)
train['piscina']=train['piscina'].astype(np.int8)
train['escuelascercanas']=train['escuelascercanas'].astype(np.int8)
train['centroscomercialescercanos']=train['centroscomercialescercanos'].astype(np.int8)
test['gimnasio']=test['gimnasio'].astype(np.int8)
test['usosmultiples']=test['usosmultiples'].astype(np.int8)
test['piscina']=test['piscina'].astype(np.int8)
test['escuelascercanas']=test['escuelascercanas'].astype(np.int8)
test['centroscomercialescercanos']=test['centroscomercialescercanos'].astype(np.int8)

#Creo categoria para los que no tienen categoria en provincia:
train['provincia']=train['provincia'].fillna('N/A')
test['provincia']=test['provincia'].fillna('N/A')
#Creo categoria para los que no tienen categoria en ciudad:
train['ciudad']=train['ciudad'].fillna('N/A')
test['ciudad']=test['ciudad'].fillna('N/A')
#Relleno los nulls en direccion con un - .
train['direccion']=train['direccion'].fillna('-')
test['direccion']=test['direccion'].fillna('-')
#Creo categoria para los que no tienen categoria en tipodepropiedad:
train['tipodepropiedad']=train['tipodepropiedad'].fillna('N/A')
test['tipodepropiedad']=test['tipodepropiedad'].fillna('N/A')

train['titulo']=train['titulo'].fillna(' ')
train['descripcion']=train['descripcion'].fillna(' ')
test['titulo']=test['titulo'].fillna(' ')
test['descripcion']=test['descripcion'].fillna(' ')

train['precio']=train['precio'].astype(np.int32)
train['id']=train['id'].astype(np.int32)
test['id']=test['id'].astype(np.int32)

train=train.fillna(0)
test=test.fillna(0)

PROBANDO CON TODOS LOS FEATURES

In [0]:
#Voy a usar OneHotEncoder para convertir la columna categorica tipodepropiedad en dummies.
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
dummies = enc.fit_transform(train['tipodepropiedad'].values.reshape(-1, 1)).toarray()
dummies = pd.DataFrame(dummies)
dummies_test = enc.transform(test['tipodepropiedad'].values.reshape(-1, 1)).toarray()
dummies_test = pd.DataFrame(dummies_test)

In [0]:
dummies.columns=enc.categories_[0].tolist()
dummies=dummies.add_prefix('prop_')
dummies=dummies.astype(np.int8)

dummies_test.columns=enc.categories_[0].tolist()
dummies_test=dummies_test.add_prefix('prop_')
dummies_test=dummies_test.astype(np.int8)

In [0]:
#Voy a usar OneHotEncoder para convertir la columna categorica provincia en dummies.
enc_prov = OneHotEncoder()
dummies_prov = enc_prov.fit_transform(train['provincia'].values.reshape(-1, 1)).toarray()
dummies_prov = pd.DataFrame(dummies_prov)

dummies_prov_test = enc_prov.transform(test['provincia'].values.reshape(-1, 1)).toarray()
dummies_prov_test = pd.DataFrame(dummies_prov_test)

In [0]:
dummies_prov.columns=enc_prov.categories_[0].tolist()
dummies_prov=dummies_prov.add_prefix('prov_')
dummies_prov=dummies_prov.astype(np.int8)

dummies_prov_test.columns=enc_prov.categories_[0].tolist()
dummies_prov_test=dummies_prov_test.add_prefix('prov_')
dummies_prov_test=dummies_prov_test.astype(np.int8)

In [0]:
#Mismo procedimiento que el anterior pero para ciudades
enc_ciud = OneHotEncoder(handle_unknown='ignore')
dummies_ciud = enc_ciud.fit_transform(train['ciudad'].values.reshape(-1, 1)).toarray()
dummies_ciud = pd.DataFrame(dummies_ciud)

dummies_ciud_test = enc_ciud.transform(test['ciudad'].values.reshape(-1, 1)).toarray()
dummies_ciud_test = pd.DataFrame(dummies_ciud_test)

dummies_ciud.columns=enc_ciud.categories_[0].tolist()
dummies_ciud=dummies_ciud.add_prefix('ciud_')
dummies_ciud=dummies_ciud.astype(np.int8)

dummies_ciud_test.columns=enc_ciud.categories_[0].tolist()
dummies_ciud_test=dummies_ciud_test.add_prefix('ciud_')
dummies_ciud_test=dummies_ciud_test.astype(np.int8)

In [0]:
#Califico las fechas de acuerdo a la diferencia entre la fecha actual y la fecha mínima sobre el máximo de dias
fecha_min = train.fecha.min()
fecha_max = train.fecha.max()
delta_max_days = (fecha_max - fecha_min).days
def calificar_fecha(fecha):
    delta = fecha - fecha_min
    return delta.days/delta_max_days

In [0]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))

In [0]:
#Set de test
fecha_min = test.fecha.min()
fecha_max = test.fecha.max()
delta_max_days = (fecha_max - fecha_min).days
test["puntaje_por_fecha"] = test['fecha'].transform(lambda x: calificar_fecha(x))

In [0]:
#Paso todas las descripciones a minúscula para contar las palabras con mas precision
train['descripcion']=train['descripcion'].apply(lambda x: x.lower())

In [0]:
Palabras_mayor_precio=['cuarto','vestidor','terraza','family','vista','bodega',\
                       'salón','estudio','jardín','room','jacuzzi','lujo','antecomedor',\
                       'tv','fiestas','juegos','estacionamiento','gimnasio','nado',\
                       'lugares','doble','desayunador','servicio','chimenea','visita',\
                       'acabado','espacio','salon','alberca','garden','spa']
Palabras_menor_precio=['minutos','acept','boiler','reja','credito','transporte'\
                      ,'cochera','contado','escuela','inf','fovissste','cerca',\
                      'bancario','protecciones','patio','infonavit']
Palabras=Palabras_mayor_precio + Palabras_menor_precio

In [0]:
dummies_palabras_descr=pd.DataFrame()
#Por cada palabra en la lista genero una columna que diga si la descripcion contenia esa palabra o no
for palabra in Palabras:
    dummies_palabras_descr[palabra]=train['descripcion'].apply(lambda x: palabra in x).astype(np.int8)
#Renombro estas columnas agregandoles un prefijo
dummies_palabras_descr=dummies_palabras_descr.add_prefix('desc_')

In [0]:
dummies_palabras_descr_test=pd.DataFrame()
#Hago lo mismo para el set de test
for palabra in Palabras:
    dummies_palabras_descr_test[palabra]=test['descripcion'].apply(lambda x: palabra in x).astype(np.int8)
dummies_palabras_descr_test=dummies_palabras_descr_test.add_prefix('desc_')

In [0]:
#Paso todas los titulos a minúscula para contar las palabras con mas precision
train['titulo']=train['titulo'].apply(lambda x: x.lower())

In [0]:
Palabras_mayor_precio=['polanco','loma','herradura','condesa','bosque','country',\
                       'hermosa','vista','residencia','condominio','golf','lujo','huixquilucan',\
                      ]
Palabras_menor_precio=['terreno','fracc','casa','villa','remate','recamaras'\
                      ,'ecatepec','cerca','coacalco','izcalli','planta','lote',\
                      'bonita','cautitlan']
Palabras=Palabras_mayor_precio + Palabras_menor_precio

In [0]:
dummies_palabras_titulo=pd.DataFrame()
#Por cada palabra en la lista genero una columna que diga si la descripcion contenia esa palabra o no
for palabra in Palabras:
    dummies_palabras_titulo[palabra]=train['titulo'].apply(lambda x: palabra in x).astype(np.int8)
#Renombro estas columnas agregandoles un prefijo
dummies_palabras_titulo=dummies_palabras_titulo.add_prefix('titl_')

In [0]:
dummies_palabras_titulo_test=pd.DataFrame()
#Hago lo mismo para el set de test
for palabra in Palabras:
    dummies_palabras_titulo_test[palabra]=test['titulo'].apply(lambda x: palabra in x).astype(np.int8)
dummies_palabras_titulo_test=dummies_palabras_titulo_test.add_prefix('titl_')

In [0]:
train['dia'] = train['fecha'].dt.day
train['mes'] = train['fecha'].dt.month
train['año'] = train['fecha'].dt.year

In [0]:
test['dia'] = test['fecha'].dt.day
test['mes'] = test['fecha'].dt.month
test['año'] = test['fecha'].dt.year

In [0]:
train['publicaciones_por_mes'] = train[['año','mes', 'id']].groupby(['año','mes']).transform(lambda x: x.count())
train['publicaciones_por_mes'] = train['publicaciones_por_mes'].fillna(0)

In [0]:
test['publicaciones_por_mes'] = test[['año','mes', 'id']].groupby(['año','mes']).transform(lambda x: x.count())
test['publicaciones_por_mes'] = test['publicaciones_por_mes'].fillna(0)

In [0]:
train['publicaciones_por_dia'] = train[['dia','año','mes', 'id']].groupby(['dia','año','mes']).transform(lambda x: x.count()).fillna(0)
test['publicaciones_por_dia'] = test[['dia','año','mes', 'id']].groupby(['dia','año','mes']).transform(lambda x: x.count()).fillna(0)

In [0]:
train['publicaciones_por_año'] = train[['año','id']].groupby(['año']).transform(lambda x: x.count()).fillna(0)
test['publicaciones_por_año'] = test[['año','id']].groupby(['año']).transform(lambda x: x.count()).fillna(0)

In [0]:
dummies_palabras=dummies_palabras_descr.join(dummies_palabras_titulo)
dummies_palabras_test=dummies_palabras_descr_test.join(dummies_palabras_titulo_test)

In [0]:
import category_encoders as ce
cat_cols = ['tipodepropiedad','ciudad', 'provincia', 'idzona']
target_enc = ce.TargetEncoder(cols = cat_cols)
target_enc.fit(train[cat_cols], train['publicaciones_por_año'])
target_encoder_publicaciones = target_enc.transform(train[cat_cols]).add_suffix('_target_publicaciones')
target_encoder_publicaciones_test = target_enc.transform(test[cat_cols]).add_suffix('_target_publicaciones')

In [0]:
cat_cols = ['tipodepropiedad','ciudad', 'provincia', 'idzona']
target_enc = ce.TargetEncoder(cols = cat_cols)
target_enc.fit(train[cat_cols], train['metroscubiertos'])
target_encoder_metros = target_enc.transform(train[cat_cols]).add_suffix('_target_metros')
target_encoder_metros_test = target_enc.transform(test[cat_cols]).add_suffix('_target_metros')

In [0]:
Numericos=['antiguedad','habitaciones','garages','banos','metroscubiertos',\
            'metrostotales','idzona', 'lat', 'lng','gimnasio','usosmultiples','piscina','escuelascercanas',\
            'centroscomercialescercanos','puntaje_por_fecha', 'publicaciones_por_año'] 

In [0]:
def preprocess_features(mexico_housing_dataframe):
 
  selected_features = mexico_housing_dataframe[Numericos]
  #DUmmies palabras no funciono en TF
  selected_features = selected_features.join(target_encoder_metros).join(target_encoder_publicaciones)
  selected_features = selected_features.rename(columns={'desc_jardín': 'desc_jardin', 'desc_salón': 'desc_salon', 'publicaciones_por_año':'publicaciones_por_anio'})
  processed_features = selected_features.copy()
  
  return processed_features

def preprocess_targets(data):
  output_targets = pd.DataFrame()
  output_targets["precio"] = (
    data["precio"] / 1000.0)
  return output_targets

In [0]:
def construct_feature_columns(input_features):
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    features = {key:np.array(value) for key,value in dict(features).items()}                                             
 
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
      ds = ds.shuffle(10000)
    
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [0]:
def train_nn_regression_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  
  periods = 10
  steps_per_period = steps / periods
  
  # Creacion de  DNNRegressor.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  dnn_regressor = tf.estimator.DNNRegressor(
      feature_columns=construct_feature_columns(training_examples),
      hidden_units=hidden_units,
      optimizer=my_optimizer
  )
  
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["precio"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["precio"], 
                                                  num_epochs=1, 
                                                  shuffle=False,
                                                  )
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["precio"], 
                                                    num_epochs=1, 
                                                    shuffle=False,
                                                    )

  
  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    dnn_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
   
    training_predictions = dnn_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = dnn_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    
    print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
    
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print("Model training finished.")

 
  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  print("Final RMSE (on training data):   %0.2f" % training_root_mean_squared_error)
  print("Final RMSE (on validation data): %0.2f" % validation_root_mean_squared_error)

  return dnn_regressor

In [0]:
mexico_housing_dataframe = train
mexico_housing_dataframe = mexico_housing_dataframe.reindex(
    np.random.permutation(mexico_housing_dataframe.index))

X = train[Numericos].join(dummies_palabras).join(target_encoder_metros).join(target_encoder_publicaciones)
X_test = test[Numericos].join(dummies_palabras_test).join(target_encoder_metros_test).join(target_encoder_publicaciones_test)

In [0]:
# Choose the first 12000 (out of 17000) examples for training.
training_examples = preprocess_features(mexico_housing_dataframe.head(160000))
training_targets = preprocess_targets(mexico_housing_dataframe.head(160000))

# Choose the last 5000 (out of 17000) examples for validation.
validation_examples = preprocess_features(mexico_housing_dataframe.tail(80000))
validation_targets = preprocess_targets(mexico_housing_dataframe.tail(80000))

In [0]:
dnn_regressor = train_nn_regression_model(
    learning_rate=0.3,
    steps=500,
    batch_size=1,
    hidden_units=[10, 10],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

In [0]:
#CONFIGURACION DE INPUTS
test["precio"] = 0
test_targets = preprocess_targets(test)
test_features = preprocess_features(test)
predict_test_input_fn = lambda: my_input_fn(
      test_features, 
      test_targets["precio"], 
      num_epochs=1, 
      batch_size=600,
      shuffle=False)

#EJECUTO PREDICCION
test_predictions = linear_regressor.predict(input_fn=predict_test_input_fn)
test_predictions = np.array([item['predictions'][0] for item in test_predictions])

#CREACION DE SUBMIT
submit = test.loc[:, ['id']]
submit['target'] = test_predictions * 1000


In [0]:
#DESCARGO SUBMIT
from google.colab import files
submit.to_csv('submit_TensorFlow_GD_RedesNeuronales_1.csv', index = False)
files.download('submit_TensorFlow_GD_RedesNeuronales_1.csv')

In [0]:
mexico_housing_test_data = test
mexico_housing_test_data["precio"] = 0
test_examples = preprocess_features(mexico_housing_test_data)
test_targets = preprocess_targets(mexico_housing_test_data)

predict_testing_input_fn = lambda: my_input_fn(test_examples, 
                                               test_targets["precio"], 
                                               num_epochs=1, 
                                               shuffle=False)

test_predictions = nn_regressor.predict(input_fn=predict_testing_input_fn)
test_predictions = np.array([item['predictions'][0] for item in test_predictions])

root_mean_squared_error = math.sqrt(
    metrics.mean_squared_error(test_predictions, test_targets))

print("Final RMSE: %0.2f" % root_mean_squared_error)